In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092361")
    password.send_keys("LP7J")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
with open(r'./class_schedules/15 March - 21 March 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [7]:
temp = []

# ---------------

with open(r'./class_schedules/15 March - 21 March 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------
c = search_courses_by_date('21', course_schedule)
c = search_courses_by_name('Warm', c)
c = search_courses_by_teacher('Una', c)
# c = search_courses_by_time('PM', c)
temp.extend(c)



courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 21 Mar Sun
-----------------------

Name: Warm Stretch 溫和伸展
Teacher: Una
Time: 10:15 - 11:15 AM




In [8]:
master_counter = 1

In [9]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [10]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [11]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [12]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [13]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =60 / 30 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',FULL,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        if (master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [14]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30)
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_linux64.zip


[WDM] - Driver has been saved in cache [/home/huohsien/.wdm/drivers/chromedriver/linux64/89.0.4389.23]


'1','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:39:05.462528'

'2','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:39:56.107770'

'3','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:40:46.813210'

'4','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:41:37.502082'

'5','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:42:28.180833'

'6','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:43:18.831249'

'7','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:44:09.521273'

'8','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:45:00.170887'

'9','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:45:50.829727'

'10','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:46:41.490553'

'11','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:47:32.141075'

'12','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:48:22.891710'

'13','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:49:13.566859'

'14','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 00:50:04.247585'

'15','Warm Stretch 溫和伸展' by 'Una',FULL,'202

'120','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:19:35.005112'

'121','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:20:25.682167'

'122','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:21:16.365873'

'123','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:22:07.033101'

'124','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:22:57.716330'

'125','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:23:48.378087'

'126','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:24:39.070833'

'127','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:25:29.748747'

'128','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:26:20.399638'

'129','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:27:11.083098'

'130','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:28:01.759631'

'131','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:28:52.434692'

'132','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:29:43.088120'

'133','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 02:30:33.746700'

'134','Warm Stretch 

'238','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 03:59:13.923496'

'239','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:00:04.588666'

'240','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:00:55.262069'

'241','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:01:45.929924'

'242','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:02:36.619215'

'243','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:03:27.281933'

'244','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:04:17.955085'

'245','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:05:08.634035'

'246','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:05:59.298302'

'247','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:06:49.962339'

'248','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:07:40.612471'

'249','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:08:31.271935'

'250','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:09:21.960535'

'251','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 04:10:12.601641'

'252','Warm Stretch 

'356','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:38:52.492530'

'357','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:39:43.155557'

'358','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:40:33.789562'

'359','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:41:24.447084'

'360','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:42:15.096063'

'361','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:43:05.783629'

'362','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:43:56.460602'

'363','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:44:47.094907'

'364','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:45:37.745499'

'365','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:46:28.421893'

'366','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:47:19.103511'

'367','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:48:09.769938'

'368','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:49:00.424116'

'369','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 05:49:51.092947'

'370','Warm Stretch 

'474','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:18:30.957179'

'475','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:19:21.640307'

'476','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:20:12.316839'

'477','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:21:02.978263'

'478','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:21:53.617569'

'479','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:22:44.273729'

'480','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:23:34.932082'

'481','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:24:25.603138'

'482','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:25:16.237437'

'483','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:26:06.927248'

'484','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:26:57.589144'

'485','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:27:48.244588'

'486','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:28:38.896333'

'487','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-03-21 07:29:29.566006'

'488','Warm Stretch 